In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from ultralytics import YOLO
import os

# 设置 Matplotlib 图片显示大小
%matplotlib inline
plt.rcParams['figure.figsize'] = (12, 8)

print("依赖库导入完成。")

In [ ]:
def identify_brand_by_color(roi_image):
    """
    输入：单车区域的裁剪图 (ROI)
    输出：品牌名称字符串
    """
    # 1. 转换到 HSV 颜色空间
    hsv = cv2.cvtColor(roi_image, cv2.COLOR_BGR2HSV)
    
    # 2. 定义颜色阈值范围
    # 哈啰蓝色范围
    lower_blue = np.array([100, 43, 46])
    upper_blue = np.array([124, 255, 255])
    
    # 美团黄色范围
    lower_yellow = np.array([20, 43, 46])
    upper_yellow = np.array([34, 255, 255])
    
    # 3. 创建二值掩膜
    mask_blue = cv2.inRange(hsv, lower_blue, upper_blue)
    mask_yellow = cv2.inRange(hsv, lower_yellow, upper_yellow)
    
    # 4. 统计像素占比
    blue_count = cv2.countNonZero(mask_blue)
    yellow_count = cv2.countNonZero(mask_yellow)
    total_count = roi_image.shape[0] * roi_image.shape[1] + 1e-5 # 防止除0
    
    # 5. 逻辑判定
    if blue_count > yellow_count and (blue_count / total_count > 0.05):
        return "Hello(Blue)"
    elif yellow_count > blue_count and (yellow_count / total_count > 0.05):
        return "Meituan(Yellow)"
    else:
        return "Bike(Common)"

In [ ]:
def process_image(image_path, model):
    # 读取图片
    img = cv2.imread(image_path)
    if img is None:
        print(f"错误：无法读取图片 {image_path}")
        return
    
    # 创建用于绘图的副本
    img_viz = img.copy()
    
    # 执行推理：classes=[1] 指定只检测自行车
    results = model.predict(img, classes=[1], conf=0.25, verbose=False)
    boxes = results[0].boxes
    
    print(f"\n>>> 开始分析图片: {os.path.basename(image_path)}")
    print(f"{'ID':<4} | {'品牌':<12} | {'置信度':<6} | {'坐标 (x1, y1, x2, y2)'}")
    print("-" * 60)
    
    for i, box in enumerate(boxes):
        # 解析坐标
        x1, y1, x2, y2 = map(int, box.xyxy[0].tolist())
        conf = box.conf[0].item()
        
        # 边界保护
        h, w, _ = img.shape
        roi = img[max(0, y1):min(h, y2), max(0, x1):min(w, x2)]
        
        # 识别品牌
        brand = "Bike"
        if roi.size > 0:
            brand = identify_brand_by_color(roi)
        
        # 打印文本结果 (任务输出)
        print(f"{i+1:<4} | {brand:<12} | {conf:.2f}   | ({x1}, {y1}, {x2}, {y2})")
        
        # 设置绘图颜色 (BGR格式)
        color = (0, 255, 0) # 默认绿
        if "Blue" in brand: color = (255, 0, 0)   # 蓝
        if "Yellow" in brand: color = (0, 255, 255) # 黄
        
        # 绘图
        cv2.rectangle(img_viz, (x1, y1), (x2, y2), color, 2)
        label = f"{brand} {conf:.2f}"
        cv2.putText(img_viz, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
        
    return img_viz

In [ ]:
# 1. 指定图片路径
# 请确保当前目录下有这张图片，或者修改为你的图片文件名
image_filename = "campus_bike.jpg" 

# 2. 运行检测
result_img = process_image(image_filename, model)

In [ ]:
if result_img is not None:
    # OpenCV 使用 BGR，Matplotlib 使用 RGB，需要转换颜色通道
    img_rgb = cv2.cvtColor(result_img, cv2.COLOR_BGR2RGB)
    
    plt.imshow(img_rgb)
    plt.axis('off') # 关闭坐标轴
    plt.title("Detection Result")
    plt.show()